In [17]:
from importlib import reload
from mpa.models import Mpa
from mpa.views import mpas_all_nogeom
from django.db.models import Q, F, Func

In [18]:
mpas = mpas_all_nogeom.filter(wdpa_id__isnull=False)
mpas.count()

13581

In [18]:
mpas.update(
    no_take_mpatlas=F('no_take'),
    no_take_area_mpatlas=F('no_take_area'),
    calc_area_mpatlas=F('calc_area'),
    calc_m_area_mpatlas=F('calc_m_area'),
    status_mpatlas=F('status'),
    status_year_mpatlas=F('status_year'),
)

13428

In [15]:
m = mpas_all_nogeom[500]
m.status_year

1971